In [1]:
# === 7) Telemetry speed and projection to s ===
# 7a) pick speed channel (m/s). If in km/h, convert.
speed_cols = [c for c in lap_df.columns if "Speed" in c]
if len(speed_cols) == 0:
    raise RuntimeError("No speed column found in telemetry. Please export 'GPS Speed' or 'Speed'.")
spd_col = speed_cols[0]

tele_v = lap_df[spd_col].to_numpy(dtype=float)
# heuristic: if values look like km/h, convert to m/s
if np.nanmedian(tele_v) > 40:  # >40 km/h typical median
    tele_v = tele_v / 3.6

# 7b) build telemetry XY from lap lat/lon (independent of centerline length)
def latlon_to_xy(lat, lon, lat0=None, lon0=None):
    """
    Convert lat/lon to local ENU meters (simple equirectangular) using first point as origin.
    """
    lat = np.asarray(lat, float)
    lon = np.asarray(lon, float)
    if lat0 is None: lat0 = float(lat[0])
    if lon0 is None: lon0 = float(lon[0])
    R = 6378137.0
    dlat = np.deg2rad(lat - lat0)
    dlon = np.deg2rad(lon - lon0)
    x = R * dlon * np.cos(np.deg2rad(lat0))
    y = R * dlat
    return x, y

# pull lat/lon from lap_df (AiM exports are usually 'GPS Latitude', 'GPS Longitude')
lat_col = next((c for c in lap_df.columns if "Latitude" in c), None)
lon_col = next((c for c in lap_df.columns if "Longitude" in c), None)
if lat_col is None or lon_col is None:
    raise RuntimeError("Telemetry CSV must include GPS Latitude/Longitude columns.")

tx, ty = latlon_to_xy(lap_df[lat_col].to_numpy(), lap_df[lon_col].to_numpy())

# 7c) project telemetry XY to centerline (KDTree) — lengths now match telemetry length
from scipy.spatial import cKDTree
tree = cKDTree(np.c_[x, y])                       # centerline points
dist, idx = tree.query(np.c_[tx, ty], k=1)        # idx: one per telemetry sample
s_tele = s[np.clip(idx, 0, len(s)-1)]             # telemetry mapped to s*

# 7d) wrap, align, sort (clean)
valid = np.isfinite(s_tele) & np.isfinite(tele_v)
s_tele = s_tele[valid]
tele_v = tele_v[valid]

L = float(s[-1] - s[0])
s_tele_wrapped = np.mod(s_tele - s_tele[0], L)

def align_offset(s_src, y_src, s_ref, y_ref, n=90):
    offs = np.linspace(0, L, n, endpoint=False)
    best_off, best_score = 0.0, -1e12
    for off in offs:
        y_on_ref = np.interp(np.mod(s_src + off, L), s_ref, y_ref)
        score = -np.mean((y_on_ref - y_src)**2)
        if score > best_score:
            best_off, best_score = off, score
    return best_off

offset = align_offset(s_tele_wrapped, tele_v, S_grid, v_opt, n=120)
s_tele_aligned = np.mod(s_tele_wrapped + offset, L)
# --- Bin telemetry onto the model S_grid ---
L = float(S_grid[-1] - S_grid[0])  # same as your track length
nbins = len(S_grid)
edges = np.linspace(0.0, L, nbins + 1)

# Ensure we're using the aligned s
s_samp = np.mod(s_tele_aligned, L)

which = np.digitize(s_samp, edges) - 1
which = np.clip(which, 0, nbins - 1)

tele_binned = np.full(nbins, np.nan)
counts = np.bincount(which, minlength=nbins)
sums   = np.bincount(which, weights=tele_v, minlength=nbins)
mask = counts > 0
tele_binned[mask] = sums[mask] / counts[mask]   # mean per bin (use median if you prefer)

# Optional light smoothing for aesthetics
from scipy.signal import savgol_filter
tele_sm = tele_binned.copy()
m = np.isfinite(tele_sm)
if m.sum() >= 11:
    tele_sm[m] = savgol_filter(tele_sm[m], 11, 2)

# --- Plot on the same S_grid ---
plt.figure(figsize=(10,5))
plt.plot(S_grid, v_kappa, label="corner cap", alpha=0.35)
plt.plot(S_grid, v_opt,   label="limit v(s)", linewidth=2)
plt.plot(S_grid, tele_sm, label="telemetry speed (binned)", linewidth=1)
plt.xlabel("s [m]"); plt.ylabel("speed [m/s]")
plt.title("Speed vs s (limit vs telemetry)")
plt.legend(); plt.grid(True, alpha=0.2)
plt.show()

print("finite binned pts:", np.isfinite(tele_binned).sum(), "of", nbins)


nbins = len(S_grid)
bins = np.linspace(0.0, L, nbins + 1)           # same coverage as S_grid
which_bin = np.digitize(s_tele_aligned, bins) - 1
which_bin = np.clip(which_bin, 0, nbins - 1)

tele_binned = np.full(nbins, np.nan)
for i in range(nbins):
    vals = tele_v[which_bin == i]
    if vals.size:
        # median is robust to spikes; use mean if you prefer
        tele_binned[i] = np.nanmedian(vals)

# Optional light smoothing so the line looks nice (Savitzky–Golay)
from scipy.signal import savgol_filter
tele_binned_smooth = tele_binned.copy()
mask = np.isfinite(tele_binned_smooth)
if mask.sum() > 9:
    tele_binned_smooth[mask] = savgol_filter(tele_binned_smooth[mask],  nine :=  nine if (nine := 9) else 9, polyorder=2)

# Plot with the *binned* telemetry on the same S_grid
plt.figure(figsize=(10,5))
plt.plot(S_grid, v_kappa, label="corner cap", alpha=0.4)
plt.plot(S_grid, v_opt,   label="limit v(s)", linewidth=2)
plt.plot(S_grid, tele_binned_smooth, label="telemetry speed (binned)", linewidth=1)
plt.xlabel("s [m]"); plt.ylabel("speed [m/s]"); plt.title("Speed vs s (limit vs telemetry)")
plt.legend(); plt.grid(True, alpha=0.2)
plt.show()
order = np.argsort(s_tele_aligned)
s_tele_sorted = s_tele_aligned[order]
tele_v_sorted = tele_v[order]

# (optional) model resampled at telemetry s
v_opt_on_tele = np.interp(s_tele_sorted, S_grid, v_opt)

# Plot
plt.figure(figsize=(10,5))
plt.plot(S_grid, v_kappa, label="corner cap", alpha=0.5)
plt.plot(S_grid, v_opt,   label="limit v(s)", linewidth=2)
plt.plot(s_tele_sorted, tele_v_sorted, label="telemetry speed", linewidth=1)
plt.legend(); plt.xlabel("s [m]"); plt.ylabel("speed [m/s]")
plt.title("Speed vs s (limit vs telemetry)")
plt.grid(True, alpha=0.2)
plt.show()


# === 8) Plots ===
plt.figure(figsize=(7,6))
plt.plot(x, y)
plt.axis('equal'); plt.title("Racing line (centerline from GPS)")
plt.xlabel("x [m]"); plt.ylabel("y [m]")
plt.show()

# === 9) g–g diagram (telemetry) ===
# Use AiM channels if present; else finite-difference fallback
lat_cols = [c for c in lap_df.columns if "LatAcc" in c]
lon_cols = [c for c in lap_df.columns if "LonAcc" in c]
if lat_cols and lon_cols:
    ay = lap_df[lat_cols[0]].to_numpy(float)
    ax = lap_df[lon_cols[0]].to_numpy(float)
    # If the magnitudes look like 'g' (<= 4..5), convert to m/s^2
    if np.nanmax(np.abs(ax)) <= 5.0 and np.nanmax(np.abs(ay)) <= 5.0:
        ax *= 9.81; ay *= 9.81
else:
    # fallback: finite-difference speed, and ay from kappa
    t = lap_df["Time"].to_numpy(float)
    dt = np.gradient(t)
    v = tele_v
    ax = np.gradient(v, dt)
    k_near = kappa[np.clip(idx, 0, len(kappa)-1)]
    ay = v*v * k_near

plt.figure(figsize=(6,6))
plt.scatter(ax/9.81, ay/9.81, s=2, alpha=0.4)
plt.axhline(0, color='k', linewidth=0.5); plt.axvline(0, color='k', linewidth=0.5)
plt.xlabel("a_x [g]"); plt.ylabel("a_y [g]"); plt.title("g–g diagram (telemetry)")
plt.grid(True, alpha=0.2); plt.gca().set_aspect('equal', 'box')
plt.show()

# === 10) Model lap time vs telemetry lap time ===
# (your S_grid should exclude duplicate endpoint; if not, drop last for time sum)
S_use = S_grid[:-1] if S_grid[-1] == S_grid[0] else S_grid
V_use = v_opt[:len(S_use)]
ds_mean = np.mean(np.diff(S_use))
t_model = np.sum(ds_mean / np.maximum(V_use, 0.1))
print(f"Model lap time: {t_model:.3f} s  |  Telemetry fastest lap: {lap_info['lap_time']:.3f} s")

NameError: name 'lap_df' is not defined

In [ ]:
print(len(S_grid), len(v_opt), len(s_tele_sorted), len(tele_v_sorted))
print(f"s_tele range: {s_tele_sorted[0]:.1f} .. {s_tele_sorted[-1]:.1f}  (L={s[-1]-s[0]:.1f})")
vmax_profile_ms  = float(np.nanmax(v_opt))     # m/s
vmax_profile_kmh = vmax_profile_ms * 3.6
print(f"Top speed from profile: {vmax_profile_kmh:.1f} km/h")

In [ ]:
from scipy.signal import butter, filtfilt
import os

def lp_filt(x, fs_hz, fc_hz=4.0, order=3):
    # simple zero-phase Butterworth
    b, a = butter(order, fc_hz/(0.5*fs_hz), btype='low')
    return filtfilt(b, a, x)

# --- infer sample rate from Time if present
if "Time" in lap_df.columns:
    t = lap_df["Time"].to_numpy(float)
    fs = 1.0/np.median(np.diff(t))
else:
    fs = 50.0  # safe default for AiM

# --- choose sources
lat_cols = [c for c in lap_df.columns if "LatAcc" in c]
lon_cols = [c for c in lap_df.columns if "LonAcc" in c]

if lat_cols and lon_cols:
    ay = lap_df[lat_cols[0]].to_numpy(float)
    ax = lap_df[lon_cols[0]].to_numpy(float)
    # convert g->m/s² if needed
    if np.nanmax(np.abs(ax)) <= 5 and np.nanmax(np.abs(ay)) <= 5:
        ax *= 9.81; ay *= 9.81
else:
    # fallback: ax from v(t), ay from v²κ
    t = lap_df["Time"].to_numpy(float)
    v = tele_v  # your velocity array
    dt = np.gradient(t)
    ax = np.gradient(v, dt)
    k_near = kappa[np.clip(idx, 0, len(kappa)-1)]
    ay = v*v*k_near

# filter to kill derivative noise
ax_f = lp_filt(ax, fs, fc_hz=4.0)
ay_f = lp_filt(ay, fs, fc_hz=4.0)

plt.figure(figsize=(6,6))
plt.scatter(ax_f/9.81, ay_f/9.81, s=2, alpha=0.4)
plt.axhline(0, color='k', lw=0.5); plt.axvline(0, color='k', lw=0.5)
plt.gca().set_aspect('equal', 'box')
plt.xlabel("a_x [g]"); plt.ylabel("a_y [g]")
plt.title("g–g diagram (telemetry, filtered)")
plt.grid(True, alpha=0.25)

# quick μ estimate from filtered data
mu_est = np.percentile(np.abs(ay_f/9.81), 95)
print(f"Estimated lateral μ (95th pct): {mu_est:.2f}")


In [ ]:
# --- PERIODICITY & GRID SANITY CHECKS ---

# Assume you have a built track object `trk` already constructed in your notebook:
# trk.s, trk.x, trk.y, trk.psi, trk.kappa are 1D arrays along centerline

import numpy as np

def _delta_wrap(a, b):
    # smallest signed difference on (-pi, pi]
    d = (a - b + np.pi) % (2*np.pi) - np.pi
    return d

def report_track_periodicity(trk):
    x, y = trk.x, trk.y
    psi, kappa, s = trk.psi, trk.kappa, trk.s
    L = s[-1]

    print(f"Samples: {len(s)}  |  Track length L ≈ {L:.3f} m")

    # 1) Position closure
    pos_err = np.hypot(x[-1] - x[0], y[-1] - y[0])
    print(f"Position closure |x(L)-x(0), y(L)-y(0)|: {pos_err:.6e} m")

    # 2) Heading closure
    dpsi = _delta_wrap(psi[-1], psi[0])
    print(f"Heading closure ψ(L)-ψ(0) (wrapped): {dpsi:.6e} rad")

    # 3) Curvature seam continuity (use small window to average near ends)
    w = 5 if len(s) > 10 else 1
    k_start = np.mean(kappa[:w])
    k_end   = np.mean(kappa[-w:])
    print(f"Curvature seam <κ(0)> vs <κ(L)>: {k_start:.6e} vs {k_end:.6e}  |  Δ={k_end-k_start:.6e}")

    # 4) ds uniformity and terminal duplication
    ds = np.diff(s)
    print(f"ds stats: min={ds.min():.6e}, median={np.median(ds):.6e}, max={ds.max():.6e}")
    if np.any(ds <= 0):
        bad = np.where(ds <= 0)[0]
        print(f"WARNING: Non-positive ds at indices {bad[:10]} ...")
    # Detect duplicated terminal sample
    if pos_err < 1e-9 and abs(ds[-1]) < 1e-12:
        print("WARNING: Detected duplicated terminal sample (zero-length last segment). Drop the last row!")

report_track_periodicity(trk)
